In [3]:
import glob
import multiprocessing
import subprocess
import os, shutil

In [11]:
docker_id = '1b00558574a6'
n_proc = 32

#-r (positional arg)
pre_dir = '/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/pre'
r_vals = glob.glob(f'{pre_dir}/*/*/R2C2_raw_reads.fastq')
# -p, -o (create custom output)
main_dir = '/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/output/main'
main_subdirs = [f'{main_dir}/' + '_'.join(x.split('/')[-3:-1]) for x in r_vals]
for x in main_subdirs:
    if os.path.exists(x):
        shutil.rmtree(x)
    os.mkdir(x)
    os.mkdir(f'{x}/tmp')
p_vals = [f'{x}/tmp' for x in main_subdirs]
o_vals = [f'{x}/css.fasta' for x in main_subdirs]
# -m, -l, -d, -c (positional args)
m_val = '/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/input/NUC.4.4.mat'
l_val = 1000
d_val = 500
c_val = '/mnt/ibm_lg/daniel_le/data/longreads/c3poa_testdata/input/config'


In [12]:
%%time
# docker command
def docker_call(r_val, p_val, o_val):
    try: 
        cmnd = ' '.join(['docker',
                         'exec',
                         docker_id,
                         'python3',
                         'C3POa.py',
                         '-r',
                         r_val,
                         '-p',
                         p_val,
                         '-o',
                         o_val,
                         '-m',
                         m_val,
                         '-l',
                         str(l_val),
                         '-d',
                         str(d_val),
                         '-c',
                         c_val,
                        ])
        output = subprocess.check_output(
            cmnd, 
            stderr=subprocess.STDOUT, 
            shell=True,
            universal_newlines=True)
    except subprocess.CalledProcessError as exc:
        print("Status : FAIL", exc.returncode, exc.output)

p = multiprocessing.Pool(processes=n_proc)
try:
    p.starmap(docker_call, zip(r_vals,
                           p_vals,
                           o_vals,
                          ))
finally:
    p.close()
    p.join()

CPU times: user 6.69 s, sys: 4.54 s, total: 11.2 s
Wall time: 1h 18min 7s


In [13]:
%%time
# cat outputs
with open(f'{main_dir}/merged_ccs.fasta', 'w') as outfile:
    for fname in o_vals:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

CPU times: user 32.2 ms, sys: 22.8 ms, total: 55 ms
Wall time: 134 ms
